In [ ]:
import pandas as pd
import MySQLdb
import MeCab
import re
import requests
from bs4 import BeautifulSoup
import tweepy
import json
from tqdm.auto import tqdm
import time
from sentence_transformers import SentenceTransformer
import xgboost as xgb
import pickle
from datetime import datetime, timedelta
import numpy as np
import pickle
import xgboost as xgb

In [ ]:
infos = json.load(open("/credential.json", "r"))

consumer_key = infos["consumer_key"]
consumer_secret = infos["consumer_secret"]
access_token = infos["access_token"]
access_token_secret = infos["access_token_secret"]
CHANNEL_ACCESS_TOKEN = infos["CHANNEL_ACCESS_TOKEN"]
me = infos["me"]

In [ ]:
con = MySQLdb.connect(user="root", passwd="root",
                               host="d2_db", db="mysql", use_unicode=True, charset="utf8mb4")

#cmd = 'echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
#path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
#                           shell=True).communicate()[0]).decode('utf-8')
m = MeCab.Tagger("-Owakati")#.format(path))
model_path = "/training_bert_japanese"
model = SentenceTransformer(model_path, show_progress_bar=False)
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
tqdm.pandas()
url = re.compile("https?://[0-9a-zA-Z-._~?&=/]+")

In [ ]:
def sendLINE(text, to=me, CHANNEL_ACCESS_TOKEN=CHANNEL_ACCESS_TOKEN):
    url = "https://api.line.me/v2/bot/message/push"
    headers = {
        "Content-Type": "application/json; charset=UTF-8",
        'Authorization': 'Bearer ' + CHANNEL_ACCESS_TOKEN
    }
    data = {"messages":[{"text": text,"type":"text"}],"to":to}
    return requests.post(url, data=json.dumps(data), headers=headers).text

In [ ]:
df_tweet = pd.read_sql("SELECT * FROM d2.tweet", con=con)
df_tweet = df_tweet[df_tweet["created_at"]>datetime.now()-timedelta(1)]

In [ ]:
bst = pickle.load(open("/model/bst_fav.pickle","rb"))

In [ ]:
sendLINE("本日ツイートされた重要なツイートです")
for text, uid in tqdm(zip(df_tweet["text"], df_tweet["user_id"]), total=df_tweet.shape[0]):
    s = url.search(text)
    if s:
        try:
            url_tweet = s.group(0)
            with requests.Session() as s:
                response = s.get(url_tweet)
                if "https://www.youtube.com" in response.url:
                        continue
                soup = BeautifulSoup(response.content, "html.parser")
                #text = re.replace("\s+"," ",text)
                #text = re.replace("\n+","\n",text)
                if "セキュリティ" in soup.__repr__() or "security" in soup.__repr__():
                    message="{0}(@{1})さんのツイートです\n\n{2}"
                    user=api.get_user(uid)
                    sendLINE(message.format(user.name, user.screen_name, text))
        except:
            pass

vectors = np.asarray(model.encode(df_tweet.text))
y_pred =bst.predict(vectors)

for pred, uid in tqdm(zip(y_pred, df_tweet["user_id"]), total=df_tweet.shape[0]):
    if pred>0.5:
        message="{0}(@{1})さんのツイートです\n\n{2}"
        user=api.get_user(uid)
        sendLINE(message.format(user.name, user.screen_name, text))